#### wav파일을 주파수 대역으로 변환하기 위해 필요한 패키지 설치

In [ ]:
!pip install librosa

#### google drive와 연결

output.wav는 2초 간격으로 128종 악기별로 50개의 음이 저장되어 있는 파일로 46개의 타악기 소리가 들어가 있다.

다운로드 링크 : https://drive.google.com/file/d/1my0VG8XBD6nJQkRj30m-1zHUaMk4IXml/view



In [ ]:
from google.colab import drive

drive.mount('/content/drive')
wav_path = '/content/drive/My Drive/Colab Notebooks/code_states/output.wav'

Mounted at /content/drive


필요한 패키지 선언

In [ ]:
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import itertools
import pandas as pd

### wav -> 주파수 대역

wav는 매 순간의 음압을 측정하여 그 수치를 저장한 형태입니다. <br/>하지만 우리는 음악의 높이와 세기를 듣기 때문에 wav파일을 음악 분석에 사용하기에 적절하지 않습니다.<br/>
따라서 변환이 필요합니다.

#### 변환 방법에는 Constant-Q 변환을 사용할 것입니다.

Constant-Q 변환은 주파수 축이 로그 단위로 변환되고<br/>
주파수에 따라 해상도가 다양하게 처리되기 때문에(저주파 - 저해상도, 고주파-고해상도) <br/>
음악을 처리하는 데에 푸리에 변환보다 유리합니다.

- spt : 스펙토그램
- ins : 악기 번호
- note : 음 높이

In [ ]:
df = pd.DataFrame(columns=['spt', 'ins', 'note'])
df

,spt,ins,note


In [ ]:
n = 0
for instrument, note in itertools.product(range(128), range(50)):
    y, sr = librosa.load(wav_path, sr=None, offset=n, duration=2.0) # n초지점부터 2초까지만 데이터를 읽어옵니다.
    n += 2
    # 데이터를 늘리기 위해 white 노이즈를 섞은 버전도 함께 변환합니다
    # 시간 대역 데이터를 옥타브당 24단계로, 총 7옥타브로 변환할 겁니다.
    
    # 0 : 원본 데이터, 1e-4 : 노이즈를 0.0001만큼 넣은 것, 1e-3은 노이즈를 0.001만큼 넣은 것
    # 데이터는 원본의 3배가 된다.
    for r in (0, 1e-4, 1e-3): # 1e-4 == 0.0001, 1e-3 == 0.001
        # librosa.cqt : audio 신호를 constant-Q 로 변환시키는 메소드
        # y : audio time series
        # sr : y의 샘플링 비율
        # hop_length : 연속적인 cqt 컬럼 사이의 샘플의 수
        # fmin에서 시작한 빈도 bin의 수
        # 옥타브마다 bins의 수
        ret = librosa.cqt(y + ((np.random.rand(*y.shape) - 0.5) * r if r else 0), sr, hop_length=1024, n_bins=24*7, bins_per_octave=24)
        # 주파수의 위상은 관심없고, 세기만 보겠으니 절대값을 취해줍니다
        ret = np.abs(ret)
        df = df.append({'spt':ret, 'ins': instrument, 'note' : 38+note}, ignore_index=True)

In [ ]:
print(df.shape)
df.tail()

(19200, 3)


,spt,ins,note
19195,"[[0.0011798030435305844, 0.0011744779711336465...",127,86
19196,"[[0.0006599579615621591, 0.0006565951988574227...",127,86
19197,"[[0.0013353681, 0.0013276788, 0.0013065297, 0....",127,87
19198,"[[0.0013388196375607902, 0.001331244084344478,...",127,87
19199,"[[0.0009992131747117146, 0.0009934555156418418...",127,87


In [ ]:
import pickle
# checkpoint 생성 ( data 저장 )
with open('audio_df.pickle', 'wb') as f:
    pickle.dump(df, f)

In [ ]:
# checkpoint 불러오기 ( 저장된 파일 확인 )
with open('/content/drive/My Drive/Colab Notebooks/code_states/audio_df.pickle', 'rb') as f:
    pickled_df = pickle.load(f)

In [ ]:
pickled_df.head()

,spt,ins,note
0,"[[0.008521897, 0.008474899, 0.008346018, 0.008...",0,38
1,"[[0.008520770494292609, 0.008473948310274565, ...",0,38
2,"[[0.008728754819538147, 0.008679076054407509, ...",0,38
3,"[[0.0032996074, 0.003287613, 0.0032563636, 0.0...",0,39
4,"[[0.0033071398188110855, 0.003295212543868543,...",0,39


In [ ]:
## 왜???
for note in range(46):
    y, sr = librosa.load(wav_path, sr=None, offset=n, duration=2.0)
    n += 2
 
    for r, s in itertools.product([0, 1e-5, 1e-4, 1e-3], range(7)):

      ret = librosa.cqt(y + ((np.random.rand(*y.shape) - 0.5) * r * s if r else 0), sr, hop_length=1024, n_bins=24 * 7, bins_per_octave=24)
      ret = np.abs(ret)
      df = df.append({'spt':ret, 'ins': note+128, 'note' : 0}, ignore_index=True)
 
    # 아래의 코드는 변환된 주파수 대역의 스펙토그램을 보여줍니다.
    # librosa.display.specshow(librosa.amplitude_to_db(np.abs(ret), ref=np.max), sr=sr, x_axis='time', y_axis='cqt_note')
    # plt.colorbar(format='%+2.0f dB')
    # plt.title('Constant-Q power spectrum')
    # plt.tight_layout()
    # plt.show()

In [ ]:
print(df.shape)
df.tail()

(20488, 3)


,spt,ins,note
20483,"[[0.0001796331605431962, 0.0001987206507602246...",173,0
20484,"[[0.0014196046084273925, 0.001411564300656717,...",173,0
20485,"[[0.000477035038555121, 0.00047842154643573686...",173,0
20486,"[[0.0018589580187521994, 0.0018567185529607514...",173,0
20487,"[[0.004577312842381108, 0.004574068757239635, ...",173,0


In [ ]:
# In case you want to create a checkpoint
with open('/content/drive/My Drive/Colab Notebooks/code_states/audio_df.pickle', 'wb') as f:
    pickle.dump(df, f)

In [ ]:
with open('/content/drive/My Drive/Colab Notebooks/code_states/audio_df.pickle', 'rb') as f:
    pickled_df = pickle.load(f)

pickled_df.shape

(20488, 3)

In [ ]:
# spt = np.array(spt, np.float32)
# ins = np.array(ins, np.int16)
# np.savez('cqt.npz', spec=spt, instr=ins)  # pickle로 저장